In [2]:
from google.cloud import bigquery
import os
import pandas as pd
import plotly.express as px

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../gcp_credentials.json'
client = bigquery.Client()

# Unified Superchain transactions summary

In [48]:
query = """
    select
      txn_date,
      network,
      count(*) as txns
    from `opensource-observer.oso.rf4_transactions_window`
    group by txn_date, network
"""
txns_results = client.query(query)

In [49]:
df_txns = txns_results.to_dataframe()
df_txns.head(1)

,txn_date,network,txns
0,2024-01-27 00:00:00+00:00,ZORA,69188


In [46]:
df_txns['txns'].sum()

290464488

In [47]:
palette = {
    'OPTIMISM': '#FF0420',
    'BASE': '#1750FA',
    'ZORA': 'indigo',
    'MODE': '#D7FF00',
    'METAL': '#C16C3B',
    'FRAX': '#000000'
}
fig = px.area(
    data_frame=df_txns, x='txn_date', y='txns', color='network', 
    color_discrete_map=palette,
    category_orders={'network': ['OPTIMISM', 'ZORA', 'MODE', 'FRAX', 'METAL', 'BASE']}
)
fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)
fig.update_traces(line=dict(width=.1))

# Tagged project address registry

In [213]:
query = """
    select distinct
      a.project_id,
      a.artifact_source,
      a.artifact_source_id,
      a.artifact_type
    from `opensource-observer.oso.int_artifacts_in_ossd_by_project` a
    join `opensource-observer.oso.projects_by_collection_v1` pbc
      on a.project_id = pbc.project_id
    where
      pbc.collection_name = 'op-retrofunding-4'
      and a.artifact_type in ('DEPLOYER', 'CONTRACT', 'REPOSITORY')
      and a.artifact_source in ('OPTIMISM', 'BASE', 'ANY_EVM', 'METAL', 'MODE', 'FRAX', 'ZORA', 'GITHUB')
"""
ossd_results = client.query(query)

In [64]:
df_ossd = ossd_results.to_dataframe()
df_ossd.head(1)

,project_id,artifact_source,artifact_source_id,artifact_type
0,DVMDNW7lcfKhTfv83r_SPJiqNz0bcQ6SoHAnrIV2ZI4=,ZORA,0xca11bde05977b3631167028862be2a173976ca11,CONTRACT


In [86]:
df_ossd[df_ossd['artifact_type'] == 'DEPLOYER']['artifact_source_id'].nunique()

424

In [87]:
df_ossd[df_ossd['artifact_type'] == 'CONTRACT']['artifact_source_id'].nunique()

9088

In [88]:
df_ossd.groupby(['artifact_source', 'artifact_type'])['artifact_source_id'].count().reset_index()

,artifact_source,artifact_type,artifact_source_id
0,ANY_EVM,DEPLOYER,280
1,BASE,CONTRACT,1234
2,BASE,DEPLOYER,105
3,FRAX,CONTRACT,2
4,GITHUB,REPOSITORY,5058
5,MODE,CONTRACT,13
6,MODE,DEPLOYER,15
7,OPTIMISM,CONTRACT,7907
8,OPTIMISM,DEPLOYER,60
9,ZORA,CONTRACT,32


# Contract discovery model

In [66]:
query = """
    select
      application_id,
      project_name,
      event_source as network,
      to_artifact_name as contract_address,
      in_ossd,
      in_agora_app,
      total_gas,
      total_addresses
    from `opensource-observer.static_data_sources.RF4_ContractMetricsByProject_20240626_Snapshot`
"""
contract_discovery_results = client.query(query)

In [214]:
df_contracts = contract_discovery_results.to_dataframe()
#df_contracts.to_parquet('data/op_rf4_contracts_by_project.parquet') # backup
df_contracts.head(1)

,application_id,project_name,network,contract_address,in_ossd,in_agora_app,total_gas,total_addresses
0,0xb98778ca9ff41446e2bc304f7b5d27f0fa7c2bcd11df...,Account Abstraction - ERC-4337,BASE,0x5ff137d4b0fdcd49dca30c7cf57e578a026d2789,True,False,197.718467,1123


In [89]:
len(df_contracts[df_contracts['in_agora_app'] == True])

1660

In [90]:
len(df_contracts[df_contracts['in_ossd'] == True])

1810

In [91]:
(
    df_contracts[(df_contracts['in_ossd'] == False) & (df_contracts['in_agora_app'] == False)]
    .groupby('project_name')['contract_address']
    .nunique()
    .sort_values()
).tail(10)

project_name
Aragon                                      112
Supswap                                     114
Synthetix                                   117
Unlonely                                    122
Reserve Protocol                            127
Decent.xyz                                  128
Ionic Protocol                              131
SwapBased                                   131
Kim Exchange                                133
Harvest Finance                             133
DackieSwap                                  138
ModeChat.xyz | Onchain Social on Mode       143
Balancer                                    164
Alien Base                                  165
Virtuals Protocol                           171
Base on Rarible                             173
Velocimeter/Graphene                        192
Gitcoin                                     203
Wormhole Foundation                         226
Arcadia Finance                             240
sudoswap                   

In [103]:
px.treemap(
    data_frame=df_contracts,
    path=['network', 'project_name'],
    values='total_gas',
    title=f'L2 Gas Fees over RF4 window by application ({df_contracts.total_gas.sum():,.0f} ETH)'
)

# Metric analysis

In [120]:
df_results = pd.read_csv('data/op_rf4_impact_metrics_by_project.csv')
df_results

,project_name,application_id,is_oss,gas_fees,transaction_count,trusted_transaction_count,trusted_transaction_share,trusted_users_onboarded,daily_active_addresses,trusted_daily_active_users,monthly_active_addresses,trusted_monthly_active_users,recurring_addresses,trusted_recurring_users,power_user_addresses,openrank_trusted_users_count,log_gas_fees,log_transaction_count,log_trusted_transaction_count
0,Aave,0x3061b642db56c507e265f03029735b0413a613bda434...,True,10.354093,622906.0,57100.0,0.091667,831.0,1263.372951,126.942623,24398.148148,2339.876543,10650.0,1777.0,24816.0,230.0,1.055152,5.794423,4.756644
1,Alchemix,0x12d482385e2c26d37da1f30c513d66c1fa39e9ed4980...,True,0.026137,4523.0,492.0,0.108777,3.0,10.000000,1.241803,206.296296,27.407407,103.0,13.0,599.0,10.0,0.011205,3.655523,2.692847
2,Biconomy,0x220685a4514003f22cbf18a7ddd9c09f88c2383daed6...,False,0.680701,3668646.0,103.0,0.000000,4.0,896.418033,0.336066,3634.444444,8.641975,386.0,3.0,109.0,4.0,0.225490,6.564506,2.017033
3,0x Protocol,0x0fad18f37ecec4f4f4ca18e60c353eb11462363e8aa1...,True,75.066083,1739387.0,88370.0,0.050805,2260.0,3728.635246,240.336066,67667.777778,3766.419753,31404.0,3502.0,24012.0,1094.0,1.881191,6.240396,4.946310
4,CreateMyToken,0xdf3eb6b429b2150c71241de0c591b18c98a3528ad0e8...,False,0.000020,3.0,0.0,0.000000,0.0,0.012295,0.000000,0.370370,0.000000,0.0,0.0,0.0,0.0,0.000009,0.602060,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,sudoswap,0x73899e48df9f895ffbffc36d53ce970c1454c8eeb070...,True,0.831091,3835.0,1677.0,0.437288,67.0,7.516393,3.684426,142.345679,63.703704,40.0,30.0,346.0,115.0,0.262710,3.583879,3.224792
226,tanX - The World's Leading Institutional DEX,0xa8213245a57be579692ead3a2381364f5942a68afd17...,False,0.005960,17607.0,5434.0,0.308627,267.0,44.606557,16.303279,1289.876543,484.074074,12.0,0.0,6790.0,44.0,0.002581,4.245710,3.735200
227,zkBob smart contract privacy wallet,0x61d2e05f988e678b240a7f00c5f722a5a69f06f085f4...,False,0.377365,18176.0,354.0,0.019476,4.0,18.225410,1.307377,380.617284,24.074074,200.0,13.0,231.0,7.0,0.139049,4.259522,2.550228
228,🏰 BuidlGuidl,0x79196c0af1a1f78fe5a009a4753c46d2271b17fc2836...,True,0.409022,25015.0,2301.0,0.091985,174.0,90.868852,7.684426,2725.925926,228.641975,9.0,2.0,1990.0,86.0,0.148918,4.398218,3.362105


In [146]:
fig = px.scatter(
    data_frame=df_results,
    x='transaction_count',
    y='trusted_transaction_count',
    color_discrete_sequence=['#FF0420'],
    custom_data=['project_name'],
    size='trusted_transaction_share',
    title=f'Transactions over RF4 window by application (n={df_results.transaction_count.sum():,.0f} txns)'
)
fig.update_traces(
    hovertemplate="<br>".join([
        "Project: %{customdata[0]}",
        "Transactions: %{x:,.0f}",
        "Trusted Transactions: %{y:,.0f}"
    ]),
    line=dict(width=.1, color='gray')
)
fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

In [149]:
fig = px.scatter(
    data_frame=df_results,
    x='transaction_count',
    y='trusted_transaction_count',
    color_discrete_sequence=['#FF0420'],
    custom_data=['project_name'],
    size='trusted_transaction_share',
    title=f'LOG SCALE: Transactions over RF4 window by application (n={df_results.transaction_count.sum():,.0f} txns)',
    log_x=True,
    log_y=True
)
fig.update_traces(
    hovertemplate="<br>".join([
        "Project: %{customdata[0]}",
        "Transactions: %{x:,.0f}",
        "Trusted Transactions: %{y:,.0f}"
    ]),
    line=dict(width=.1, color='gray')
)
fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)

In [176]:
dff = df_results.sort_values(by='openrank_trusted_users_count', ascending=False).head(40)
fig = px.bar_polar(
    data_frame=dff,
    r='openrank_trusted_users_count',
    theta='project_name',
    color_discrete_sequence=['#FF0420'],
    title='Top 40 applications by Badgeholder OpenRank'
)
fig.update_layout(
    polar = dict(
            bgcolor = "white",
            angularaxis = dict(tickfont = dict(size=11)),            
            radialaxis = dict(showticklabels=False, ticks='', gridcolor = 'black')
        )
)

# Backup: check for duplicate contracts

In [ ]:
dupes = df_contracts.groupby(['contract_address', 'network'])['project_name'].nunique().sort_values()
dupes = dupes[dupes>1]
print((len(dupes) / len(df_contracts)) * 100)
(
    df_contracts
    .set_index(['contract_address', 'network'])
    .loc[dupes.index, ['project_name', 'total_gas']]
    .sort_values(by='total_gas', ascending=False)
)